In [ ]:
!pip install efficientnet
!pip install nnAudio

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
from scipy.signal import butter,lfilter
from scipy.fft import fft,ifft
import pandas as pd
import os
import glob
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
import datetime
import time
#from tensorflow.keras.applications import EfficientNetB0
from efficientnet.keras import EfficientNetB0
import torch
from tensorflow.keras.utils import Sequence
import math
from random import shuffle
import tensorflow.keras.layers as L
import pywt

In [ ]:
targets = pd.read_csv('../input/g2net-gravitational-wave-detection/training_labels.csv')
sample_submission = pd.read_csv('../input/g2net-gravitational-wave-detection/sample_submission.csv')

In [ ]:
fs = 2048
steps_per_second = 32
upsamp = fs//steps_per_second
min_width = max((0.1,steps_per_second/500))
max_width = steps_per_second/20
widths = np.arange(min_width, max_width, (max_width-min_width)/64)
n_samples = 100000

def get_signal_id(i):
    return targets.iloc[i]['id']

def get_path(id_no,is_train=True):
    file_path = "../input/g2net-gravitational-wave-detection/{}/{}/{}/{}/{}.npy"
    if is_train:
        path = file_path.format('train',id_no[0],id_no[1],id_no[2],id_no)
    else:
        path = file_path.format('test',id_no[0],id_no[1],id_no[2],id_no)
    return path    

def get_signal(id_no,is_train=True):
    file = get_path(id_no,is_train)
    signal = np.load(file)
    return signal

def get_signals(id_no,is_train=True):
    file = get_path(id_no,is_train)
    signal = np.load(file)
    return np.hstack(signal)

def get_filtered_scaled_signals(id_no,is_train=True):
    signal = get_signal(id_no,is_train)   
    return np.hstack(filter_scale_signals(signal))

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def filter_scale_signal(signal):
    tmp = butter_bandpass_filter(signal, 20, 500, fs, order=5)
    tmp = 2*(tmp - min(tmp))/(max(tmp)-min(tmp))-1
    return tmp

def filter_scale_signals(signal):
    return [filter_scale_signal(s) for s in signal]

def get_scalogram(id_no,is_train):    
    sigs = get_filtered_scaled_signals(id_no,is_train)
    cwts = sig.cwt(sigs[0:-1:upsamp], sig.ricker, np.arange(1,65))
    #cwts,freqs = pywt.cwt(sigs[:-1:upsamp],np.arange(1,65),'morl',sampling_period=1/2048)
    return cwts.reshape((cwts.shape[0],cwts.shape[1],1))#out_mat = np.zeros((cwts.shape[0],cwts.shape[1]))

def get_model(shape):
    model = tf.keras.Sequential([
        L.InputLayer(input_shape=(shape[0],shape[1],1)),
        L.Conv2D(3,3,activation='relu',padding='same'),
        EfficientNetB0(include_top=False,input_shape=(),weights='imagenet'),
        L.GlobalAveragePooling2D(),
        L.Dense(32,activation='relu'),
        L.Dense(1, activation='sigmoid')])

    model.summary()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC()])
    return model

from nnAudio.Spectrogram import CQT1992v2
def get_spectrogram(idx,is_train,transform=CQT1992v2(sr=2048, fmin=20, fmax=1024, hop_length=64)): # in order to use efficientnet we need 3 dimension images
    waves = np.load(get_path(idx,is_train))
    waves = np.hstack(waves)
    waves = waves / np.max(waves)
    waves = torch.from_numpy(waves).float()
    image = transform(waves)
    image = np.array(image)
    image = np.transpose(image,(1,2,0))
    #image = np.array(get_scalogram(idx,is_train))
    return image

class Dataset(Sequence):
    def __init__(self,idx,y=None,batch_size=256,shuffle=True):
        self.idx = idx
        self.batch_size = batch_size
        self.shuffle = shuffle
        if y is not None:
            self.is_train=True
        else:
            self.is_train=False
        self.y = y
    def __len__(self):
        return math.ceil(len(self.idx)/self.batch_size)
    def __getitem__(self,ids):
        batch_ids = self.idx[ids * self.batch_size:(ids + 1) * self.batch_size]
        if self.y is not None:
            batch_y = self.y[ids * self.batch_size: (ids + 1) * self.batch_size]
            
        list_x = np.array([get_spectrogram(x,self.is_train) for x in batch_ids])
        batch_X = np.stack(list_x)
        if self.is_train:
            return batch_X, batch_y
        else:
            return batch_X
    
    def on_epoch_end(self):
        if self.shuffle and self.is_train:
            ids_y = list(zip(self.idx, self.y))
            shuffle(ids_y)
            self.idx, self.y = list(zip(*ids_y))

In [ ]:
f, t, Sxx = sig.spectrogram(get_filtered_scaled_signals(get_signal_id(0)), fs)
plt.pcolormesh(t, f, Sxx, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.ylim(1,100)
plt.show()
Sxx.shape

In [ ]:
plt.plot(get_signals(get_signal_id(0)))

In [ ]:
plt.plot(get_filtered_scaled_signals(get_signal_id(0)))

In [ ]:
plt.plot(filter_scale_signal(get_signals(get_signal_id(0))))

In [ ]:
cwtmatr = get_scalogram(get_signal_id(0),True)
plt.imshow(cwtmatr,aspect='auto')
plt.show()

cwtmatr.shape

In [ ]:
cwtmatr = get_scalogram(get_signal_id(1),True)
plt.imshow(cwtmatr,aspect='auto')
plt.show()

cwtmatr.shape

In [ ]:
coefs,freqs = pywt.cwt(get_signals(get_signal_id(0))[:-1:64],np.arange(1,65),'morl',sampling_period=1/2048)
fig = plt.figure()
ax = fig.gca()
plt.imshow(coefs,aspect='auto')
ax.set_yticklabels(freqs)
plt.show()

In [ ]:
coefs,freqs = pywt.cwt(get_filtered_scaled_signals(get_signal_id(0))[:-1:64],np.arange(1,65),'morl',sampling_period=1/2048)
fig = plt.figure()
ax = fig.gca()
plt.imshow(coefs,aspect='auto')
ax.set_yticklabels(freqs)
plt.show()

In [ ]:
spect = get_spectrogram(get_signal_id(0),True)
plt.imshow(spect)
spect.shape

In [ ]:
plt.plot(np.log(abs(fft(get_filtered_scaled_signals(get_signal_id(0))))))
plt.xlim(15,550)
plt.show()

In [ ]:
plt.plot(np.log(abs(fft(get_filtered_scaled_signals(get_signal_id(1))))))
plt.xlim(15,550)
plt.show()

In [ ]:
samples = targets#.sample(20000)
train_idx =  samples['id'].values
y = samples['target'].values
test_idx = sample_submission['id'].values
x_train,x_valid,y_train,y_valid = train_test_split(train_idx,y,test_size=0.05,random_state=42,stratify=y)

train_dataset = Dataset(x_train,y_train)
valid_dataset = Dataset(x_valid,y_valid)
test_dataset = Dataset(test_idx)

model = get_model((69,193))

In [ ]:
model.fit(train_dataset,
          epochs=1,
          validation_data=valid_dataset)

In [ ]:
preds = model.predict(test_dataset)

In [ ]:
df = pd.DataFrame({'id':sample_submission['id'],'target':preds.reshape(-1)})
df.to_csv('submission.csv',index=False)